In [1]:
import pandas as pd
import datetime as dt

In [2]:
drop_cols = ['replies_count', 'retweets_count', 'likes_count', 'cashtags', 'conversation_id',
            'language', 'mentions', 'urls', 'photos', 'quote_url' ,'video' ,'thumbnail' ,'near',
            'geo', 'source', 'user_rt_id', 'retweet_id', 'reply_to', 'retweet_date', 'created_at',
            'translate', 'trans_src', 'trans_dest', 'place', 'name', 'timezone', 'user_rt', 'link', 'retweet']


In [3]:
df_biden = pd.read_csv("../data/tweets/all_biden_2020-11-02.csv").drop(columns=drop_cols)
df_trump = pd.read_csv("../data/tweets/all_trump_2020-11-02.csv").drop(columns=drop_cols)


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_trump['date'] = pd.to_datetime(df_trump['date'], format="%Y-%m-%d")
df_trump['time'] = pd.to_datetime(df_trump['time'],format='%H:%M:%S').dt.time
df_biden['date'] = pd.to_datetime(df_biden['date'], format="%Y-%m-%d")
df_biden['time'] = pd.to_datetime(df_biden['time'],format='%H:%M:%S').dt.time

df_trump.head(3)

,id,date,time,user_id,username,tweet,hashtags
0,1323414586586968064,2020-11-02,23:59:59,1242220312676241409,kamalatoe,Trump is campaigning with steel and autoworker...,"['macon', 'gasen', 'savannah', 'riseup']"
1,1323414586511429634,2020-11-02,23:59:59,2324659466,didih2214,Trump is having unscalable fencing erected aro...,['votehimout2020']
2,1323414586503143424,2020-11-02,23:59:59,215506541,cuffy__,"They support trump like he cares about them, h...",[]


In [5]:

dtypes = {
    'id': str,
    'user_id': str,
    'username': str,
    'tweet': str
}

df_trump = df_trump.astype(dtypes)
df_biden = df_biden.astype(dtypes)

In [6]:
df_biden.dtypes

id                  object
date        datetime64[ns]
time                object
user_id             object
username            object
tweet               object
hashtags            object
dtype: object

In [7]:
df_nov2 = pd.concat((pd.DataFrame(), df_trump, df_biden))

In [8]:
df_nov2.sample(10)

,id,date,time,user_id,username,tweet,hashtags
343144,1323413066382540801,2020-11-02,23:53:57,248365984,mrkrest,@iamtenseven @Mr_Nnaji_ @Nedunaija @realDonald...,[]
192561,1323376896063672321,2020-11-02,21:30:13,49001404,mismahvash,"@realDonaldTrump Hey useless, you are losing! ...",[]
76180,1323402082146242561,2020-11-02,23:10:18,1251495142357426176,sbwheat1,"@ToddKIL @GeorgeTakei Well, Todd, you'll be ev...",[]
64183,1323395809585627142,2020-11-02,22:45:23,1016060761263747072,abdullsmith,"@JoeBiden Ansioso pela sua vitória, presidente...",[]
347555,1323413947811303426,2020-11-02,23:57:27,1073256492504137728,newrozhassan,@sabine_mueller1 Trump 2020 Auf dem Grabstein🥰,[]
52034,1323407384111448066,2020-11-02,23:31:22,161104263,thelyricalnerd,Biden/Harris are no more different than Trump/...,[]
20014,1323414465606520835,2020-11-02,23:59:31,1276115805969821697,douglas77715,@NateSilver538 @ForecasterEnten Am so sick of ...,[]
215651,1323372008974016512,2020-11-02,21:10:48,18868110,kristinakay,@AmandaSeitz @APFactCheck Trump supporters wil...,"['democrats', 'voteblue']"
160944,1323383588897935362,2020-11-02,21:56:49,19625776,dmsalcedo,"Trump, Venezuela and the Tug-of-War Over a Str...",[]
21440,1323414142431170566,2020-11-02,23:58:14,1321935983194066946,aunt_tiffy,Humpty Dumpty tRump attacking Jon Bon Jovi is ...,[]


In [9]:
df_nov2.drop_duplicates(subset='id', inplace=True)
df_nov2.shape

(359698, 7)

In [10]:
df_nov2['trump'] = df_nov2.tweet.str.contains("[Tt][Rr][Uu][Mm][Pp]")
df_nov2['biden'] = df_nov2.tweet.str.contains("[Bb][Ii][Dd][Ee][Nn]")
df_nov2.head(3)

,id,date,time,user_id,username,tweet,hashtags,trump,biden
0,1323414586586968064,2020-11-02,23:59:59,1242220312676241409,kamalatoe,Trump is campaigning with steel and autoworker...,"['macon', 'gasen', 'savannah', 'riseup']",True,False
1,1323414586511429634,2020-11-02,23:59:59,2324659466,didih2214,Trump is having unscalable fencing erected aro...,['votehimout2020'],True,False
2,1323414586503143424,2020-11-02,23:59:59,215506541,cuffy__,"They support trump like he cares about them, h...",[],True,False


In [11]:
df_nov2.trump.value_counts()

True     285919
False     73779
Name: trump, dtype: int64

In [12]:
df_nov2.biden.value_counts()

False    221981
True     137717
Name: biden, dtype: int64

## Filtering Our Data Down

We need to do two things:
1. Exclude tweets with under 70 characters.
2. We'll only use tweets that mention one candidate; and not the other
    * equal amount of each

In [13]:
# exclude tweets with under 70 characters

mask = df_nov2.tweet.str.len() > 70
df_nov2 = df_nov2[mask]
df_nov2.shape

(280373, 9)

In [17]:
mask = (df_nov2.trump == 0) & (df_nov2.biden == 1)
biden_72k = df_nov2[mask].head(48000)

mask = (df_nov2.trump == 1) & (df_nov2.biden == 0)
trump_72k = df_nov2[mask].head(48000)

df_balanced = pd.concat((pd.DataFrame(), biden_72k, trump_72k))
df_balanced

,id,date,time,user_id,username,tweet,hashtags,trump,biden
181142,1323379284434669568,2020-11-02,21:39:43,2820503362,artistacriseida,All these articles showing that Biden is in th...,[],False,True
0,1323414585995526144,2020-11-02,23:59:59,1312487180258820096,annapieters17,@FoxNews Lady Gaga’s a nobody. Can’t figure ou...,[],False,True
4,1323414585232293888,2020-11-02,23:59:59,2335763630,kylechwatt,@The_Grupp “It is purely a fortuity that this ...,[],False,True
5,1323414585097908224,2020-11-02,23:59:59,3282973915,medianpolitics,@MSNBC What? When words just don't match rea...,[],False,True
8,1323414583483256834,2020-11-02,23:59:59,100625142,michaelmrw,Biden in his Union if I win is just revolting ...,[],False,True
...,...,...,...,...,...,...,...,...,...
96806,1323397528042299393,2020-11-02,22:52:12,85100748,paloma_roque,Trump no construyó esas jaulas en la frontera ...,[],True,False
96809,1323397527488659457,2020-11-02,22:52:12,1759742816,mrhumblebe,@sidounours @gudulus prix Nobel de la paix san...,[],True,False
96810,1323397527404687361,2020-11-02,22:52:12,232179414,nepcar,@mkraju After this guy - like the other GOPs -...,[],True,False
96811,1323397527316606977,2020-11-02,22:52:12,392367257,caguan2407,Los votantes latinos constituyen el 15 % del p...,[],True,False


In [18]:
df_balanced.to_pickle("pickle/balanced_nov2_tweets.pick")